In [1]:
# !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

In [2]:

# # Установка остальных пакетов
# %pip install ultralytics
# %pip install opencv-python==4.8.0.74
# %pip install wandb

In [3]:
# Исправление проблемы с кодировкой для subprocess
# Устанавливаем переменные окружения ПЕРЕД импортом numpy/scipy
import os
import sys

# Установка кодировки UTF-8 для subprocess (исправляет UnicodeDecodeError)
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'
os.environ['PYTHONIOENCODING'] = 'utf-8'

# Принудительная установка кодировки для Python
if hasattr(sys.stdout, 'reconfigure'):
    sys.stdout.reconfigure(encoding='utf-8')
if hasattr(sys.stderr, 'reconfigure'):
    sys.stderr.reconfigure(encoding='utf-8')

print("Кодировка установлена для subprocess")


Кодировка установлена для subprocess


In [4]:
# !pip install "numpy<2.0" "numpy>=1.23.0"


In [5]:
from ultralytics import YOLO
import torch

# Проверка доступности CUDA
print(f"PyTorch версия: {torch.__version__}")
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA версия: {torch.version.cuda}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Текущее GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("CUDA недоступна, будет использоваться CPU (тренировка будет медленнее)")

PyTorch версия: 2.5.1+cu121
CUDA доступна: True
CUDA версия: 12.1
Количество GPU: 1
Текущее GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Память GPU: 7.62 GB


In [6]:
# Установка рабочей директории (фиксированный путь проекта)

import os
import sys
from pathlib import Path

# Установка кодировки UTF-8 для корректной работы с кириллицей
os.environ['PYTHONIOENCODING'] = 'utf-8'
os.environ['LANG'] = 'ru_RU.UTF-8'
os.environ['LC_ALL'] = 'ru_RU.UTF-8'
# Принудительная установка кодировки для Python
if hasattr(sys.stdout, 'reconfigure'):
    sys.stdout.reconfigure(encoding='utf-8')
if hasattr(sys.stderr, 'reconfigure'):
    sys.stderr.reconfigure(encoding='utf-8')

# Выбор предобученной модели YOLOv8
# - 'yolov8n.pt' - nano 
# - 'yolov8s.pt' - small
# - 'yolov8m.pt' - medium
# - 'yolov8l.pt' - large
# - 'yolov8x.pt' - extra large
# имя модели задаём здесь

model_name = 'yolov8s.pt'

PROJECT_ROOT = Path("/home/gleb/Документы/33.VKR/mvp/road_monitoring").resolve()
os.chdir(PROJECT_ROOT)

print(f"Текущая директория: {os.getcwd()}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Кодировка: {sys.stdout.encoding}")

# Пути для моделей
TRAINING_DIR = PROJECT_ROOT / "training"
PRETRAINED_MODELS_DIR = TRAINING_DIR / "pretrained_models"

# Путь к модели
model_path = "training/pretrained_models/yolov8s.pt"



Текущая директория: /home/gleb/Документы/33.VKR/mvp/road_monitoring
PROJECT_ROOT: /home/gleb/Документы/33.VKR/mvp/road_monitoring
Кодировка: UTF-8


In [7]:
#Настройки обучения

# Путь сохранения результатов обучения
_project = str(TRAINING_DIR / "runs" / "rdd_split")
_name = model_name[0:-3]

# Путь к YAML
_data = str(TRAINING_DIR / "rdd_split.yaml")


# Количество эпох (полных проходов по датасету)
_epoch = 100

# Warmup эпохи - постепенное увеличение learning rate в начале тренировки
_warmup_epochs = 5  # 

# Автоматический выбор: CUDA если доступна, иначе CPU
_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {_device}")
print("Автовзвешивание классов: включено (YOLOv8)")



model = YOLO(model_path)


print(f"\n{'='*50}")
print(f"НАЧАЛО ТРЕНИРОВКИ")
print(f"{'='*50}")
print(f"Проект: {_project}")
print(f"Эксперимент: {_name}")
print(f"Датасет: {_data}")
print(f"Эпох: {_epoch}")
print(f"Warmup эпох: {_warmup_epochs}")
print(f"Устройство: {_device}")
print(f"{'='*50}\n")

results = model.train(
    # Путь к YAML файлу с описанием датасета
    data=_data,
    
    # Количество эпох
    epochs=_epoch,
    
    # Количество эпох для warmup (постепенный разогрев learning rate)
    warmup_epochs=_warmup_epochs,
    
    # Размер батча (количество изображений за одну итерацию)

    batch=32,
    
    lr0=0.001,
    lrf=0.01,
    
    # Размер входного изображения (пиксели) 
    imgsz=640,
    
    # Сохранять чекпоинт каждые N эпох 
    save_period=5,
    
    # Количество воркеров для загрузки данных 
    workers=4,
    
    # Устройство для вычислений (cuda/cpu)
    device=_device,
    
    # Директория для сохранения результатов
    project=_project,
    
    # Имя эксперимента (поддиректория в project)
    name=_name,
    
    # Seed для воспроизводимости результатов
    seed=1337,
    
    # Cosine learning rate scheduler
    cos_lr=True,
    
    # Mosaic augmentation (0.0 = отключено)
    # Смешивание 4 изображений в одно 
    mosaic=0.3,
    close_mosaic=5,

    # Масштабирование (scale=0.2 даёт диапазон 0.8-1.2)
    # Диапазон изменения размера: 1.0 ± scale
    scale=0.2,
    
    # Поворот изображений (±5 градусов)
    degrees=5.0,
    
    mixup=0.1,
    copy_paste=0.1,
    
    patience=10,
    
    # cls=0.5,  # Можно не указывать, используется по умолчанию
    

    # Сохранение графиков и визуализаций (True по умолчанию)
    # Создает: results.png, confusion_matrix.png, F1_curve.png, PR_curve.png и др.
    plots=True,
    
    # Подробные логи в консоль (True по умолчанию)
    verbose=True
)


print(f"\n{'='*50}")
print(f"ТРЕНИРОВКА ЗАВЕРШЕНА!")
print(f"Результаты сохранены в: {_project}/{_name}/")
print(f"{'='*50}")

Используемое устройство: cuda
Автовзвешивание классов: включено (YOLOv8)

НАЧАЛО ТРЕНИРОВКИ
Проект: /home/gleb/Документы/33.VKR/mvp/road_monitoring/training/runs/rdd_split
Эксперимент: yolov8s
Датасет: /home/gleb/Документы/33.VKR/mvp/road_monitoring/training/rdd_split.yaml
Эпох: 100
Warmup эпох: 5
Устройство: cuda

New https://pypi.org/project/ultralytics/8.4.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.8 🚀 Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=5, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/gleb/Документы/33.VKR/mvp/road_monitoring/training/rdd_split.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs

In [8]:
# Продолжение тренировки (если нужно возобновить с последнего чекпоинта)
#
# По умолчанию Ultralytics сохраняет сюда: <project>/<name>/weights/last.pt
# weight_path = str(Path(_project) / _name / "weights" / "last.pt")
# 
# model_Trained = YOLO(weight_path)
# results = model_Trained.train(
#     resume=True,
#     device='cuda' if torch.cuda.is_available() else 'cpu'  # Использование CUDA
# )